TODO:

0) automatize corr & best setups displaing
1) ROSE LightGBM
2) SMOTE RF, XGB, LightGBM
3) Weights RF, LightGBM

4) Compare models' performances (df + few plots + conclusion)

In [112]:
import re
import os
import pandas as pd
from itertools import product
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from lightgbm import LGBMClassifier

from tools.clf_hyperparameters import run_inference, generate_hprs_report

ImportError: cannot import name 'generate_hprs_report' from 'tools.clf_hyperparameters' (/home/maksym/Documents/Projects/Taiwan-Companies-Bankruptcy-Prediction/tools/clf_hyperparameters.py)

# Load data

In [99]:
df_rose = pd.read_csv('data/.local/ROSE_data.csv')
df_smote = pd.read_csv('data/.local/SMOTE_data.csv')
df_test = pd.read_csv('data/.local/TEST_data.csv')
df_train = pd.read_csv('data/.local/TRAIN_data.csv')

In [39]:
df_smote.head()

,operating gross margin,operating profit rate,tax rate (a),cash flow per share,revenue per share (yuan ¥),realized sales gross profit growth rate,operating profit growth rate,regular net profit growth rate,continuous net profit growth rate,total asset return growth rate ratio,...,cfo to assets,cash flow to equity,current liability to current assets,net income to total assets,total assets to gnp price,no-credit interval,degree of financial leverage (dfl),interest coverage ratio (interest expense to ebit),equity to liability,bankrupt
0,0.620440,0.999212,0.008807,0.348942,0.029055,0.022110,0.848043,0.689519,0.217607,0.264392,...,0.644730,0.329472,0.010759,0.834316,0.003665,0.625232,0.026800,0.565199,0.080997,0
1,0.613932,0.999043,0.227452,0.320365,0.015715,0.022093,0.848022,0.689338,0.217585,0.263891,...,0.590121,0.316132,0.013225,0.808498,0.000760,0.623952,0.026899,0.565624,0.054215,0
2,0.609132,0.998946,0.000000,0.321178,0.004810,0.022295,0.848148,0.689709,0.217629,0.265260,...,0.595232,0.314551,0.072953,0.797465,0.001693,0.623293,0.070125,0.570077,0.041042,0
3,0.589840,0.998842,0.000000,0.317429,0.004174,0.022141,0.848053,0.689427,0.217602,0.263962,...,0.563640,0.316418,0.058115,0.779479,0.006058,0.624777,0.026617,0.564167,0.029985,0
4,0.605385,0.999054,0.106387,0.309295,0.037010,0.022118,0.848035,0.689294,0.217598,0.263822,...,0.532495,0.311484,0.025038,0.803455,0.001516,0.624009,0.027180,0.566515,0.018928,0


In [104]:
X_rose = df_rose[df_rose.columns[:-1]].to_numpy()
Y_rose = df_rose['bankrupt'].to_numpy()
print("\tROSE sample:\n", pd.Series(Y_rose).value_counts().values)

X_smote = df_smote[df_smote.columns[:-1]].to_numpy()
Y_smote = df_smote['bankrupt'].to_numpy()
print("\tSMOTE sample:\n", pd.Series(Y_rose).value_counts().values)

X_train = df_train[df_train.columns[:-1]].to_numpy()
Y_train = df_train['bankrupt'].to_numpy()
print("\tTrain sample:\n", pd.Series(Y_train).value_counts().values)

X_test = df_test[df_test.columns[:-1]].to_numpy()
Y_test = df_test['bankrupt'].to_numpy()
print("\tTest sample:\n", pd.Series(Y_test).value_counts().values)

	ROSE sample:
 [4567 4567]
	SMOTE sample:
 [4567 4567]
	Train sample:
 [4567  141]
	Test sample:
 [1948   71]


# Random Forest

Hyperparameters

In [82]:
HPRS = ['n_estimators', 'max_depth', 'min_samples_leaf', 'max_features']

In [86]:
rf_params1 = {
    'n_estimators': [10, 40, 80],
    'criterion': ['entropy'],
    'max_depth': [5, 20, 60],
    'min_samples_leaf': [2, 4, 6, 10],
    'max_features': [0.25, 0.5, 0.75]
}

In [87]:
rf_params2 = {
    'n_estimators': [10, 40, 80, 120, 200],
    'criterion': ['entropy'],
    'max_depth': [3, 20, 40, 80, 120],
    'min_samples_leaf': [2, 4, 6, 8, 10, 12],
    'max_features': [0.1, 0.15, 0.25, 0.4]
}

In [90]:
print('1st inference for demonstration purpose - compare iterative and multiprocessing approaches\n')
print("\n".join(re.split('(?<=]),', rf_params1.__str__())))
print("\nNumber of settups:", len(list(product(*list(rf_params1.values())))))

1st inference for demonstration purpose - compare iterative and multiprocessing approaches

{'n_estimators': [10, 40, 80]
 'criterion': ['entropy']
 'max_depth': [5, 20, 60]
 'min_samples_leaf': [2, 4, 6, 10]
 'max_features': [0.25, 0.5, 0.75]}

Number of settups: 108


In [91]:
print('2st (main) inference \n')
print("\n".join(re.split('(?<=]),', rf_params2.__str__())))
print("\nNumber of settups:", len(list(product(*list(rf_params2.values())))))

2st (main) inference 

{'n_estimators': [10, 40, 80, 120, 200]
 'criterion': ['entropy']
 'max_depth': [3, 20, 40, 80, 120]
 'min_samples_leaf': [2, 4, 6, 8, 10, 12]
 'max_features': [0.1, 0.15, 0.25, 0.4]}

Number of settups: 600


Iterative

In [59]:
run_inference(data=(X_rose, X_test, Y_rose, Y_test),
              base_clf=RandomForestClassifier,
              params=rf_params1,
              metrics=('f1-score', 'precision', 'recall'),
              classes=(0, 1),
              output_file=os.path.join('data', '.local', 'RF_it_results.csv'),
              multiprocessing_mode=False)

Number of classifiers to be trained: 108


Iterative method: 100%|██████████| 108/108 [02:49<00:00,  1.57s/it]

@timer:<function iterative at 0x7eff4c8813f0> exec. time: 169.55 s.


Multiprocessing

In [60]:
run_inference(data=(X_rose, X_test, Y_rose, Y_test),
              base_clf=RandomForestClassifier,
              params=rf_params1,
              metrics=('f1-score', 'precision', 'recall'),
              classes=(0, 1),
              output_file=os.path.join('data', '.local', 'RF_mp_results.csv'),
              multiprocessing_mode=True)

Number of classifiers to be trained: 108


Multiprocessing method: 100%|██████████| 108/108 [00:23<00:00,  4.52it/s]

Results writen into data/.local/RF_mp_results.csv.
@timer:<function processes at 0x7eff4c881510> exec. time: 24.11 s.


Main inference

# ROSE

In [61]:
run_inference(data=(X_rose, X_test, Y_rose, Y_test),
              base_clf=RandomForestClassifier,
              params=rf_params2,
              metrics=('f1-score', 'precision', 'recall'),
              classes=(0, 1),
              output_file=os.path.join('data', '.local', 'RF_ROSE_results.csv'),
              multiprocessing_mode=True)

Number of classifiers to be trained: 600


Multiprocessing method: 100%|██████████| 600/600 [02:06<00:00,  4.76it/s]

Results writen into data/.local/RF_mp_results.csv.
@timer:<function processes at 0x7eff4c881510> exec. time: 126.28 s.


In [5]:
rf_rose_res = pd.read_csv(os.path.join('data', '.local', 'RF_ROSE_results.csv'), sep=';')
rf_rose_res

,n_estimators,criterion,max_depth,min_samples_leaf,max_features,f1-score_0,f1-score_1,precision_0,precision_1,recall_0,recall_1
0,10,entropy,3,2,0.10,0.903873,0.231626,0.995092,0.133676,0.827973,0.866667
1,10,entropy,3,6,0.10,0.903873,0.231626,0.995092,0.133676,0.827973,0.866667
2,10,entropy,3,4,0.10,0.903873,0.231626,0.995092,0.133676,0.827973,0.866667
3,10,entropy,3,8,0.10,0.903873,0.231626,0.995092,0.133676,0.827973,0.866667
4,10,entropy,3,6,0.15,0.923119,0.266319,0.994693,0.157895,0.861154,0.850000
...,...,...,...,...,...,...,...,...,...,...,...
595,200,entropy,120,6,0.40,0.983172,0.431034,0.982170,0.446429,0.984176,0.416667
596,200,entropy,120,12,0.25,0.980483,0.472222,0.986563,0.404762,0.974477,0.566667
597,200,entropy,120,8,0.40,0.982097,0.453125,0.984111,0.426471,0.980092,0.483333
598,200,entropy,120,10,0.40,0.982070,0.477612,0.985604,0.432432,0.978560,0.533333


In [60]:
def generate_hprs_report(df: pd.DataFrame, hprs: list, metrics=None):
    print("\t\tHyperparameters Report")
    if metrics is None:
        metrics = ['f1-score_1']

    print("\n\tCorrelation between given metrics and hyperparameters")
    for metric in metrics:
        print(f"\n\t---{metric}---\n")
        
        m_q50 = df[metric].quantile(0.5)
        r_ = pd.DataFrame(df[df[metric] >= m_q50][hprs+[metric]].corr()[metric])
        r_ = r_.drop(metric)
        print(r_)

    print("\n\n\tBest setups per metric")
    for metric in metrics:
        print(f"\n\t---{metric}---\n")
        
        mcopy = metrics.copy()
        mcopy.remove(metric)
        r_ = pd.DataFrame(df[[metric] + hprs + mcopy].sort_values(by=metric, ascending=False))
        print(r_.head())

In [62]:
generate_hprs_report(df=rf_rose_res, hprs=HPRS, metrics=['f1-score_1', 'precision_1', 'recall_1'])

		Hyperparameters Report

	Correlation between given metrics and hyperparameters

	---f1-score_1---

                  f1-score_1
n_estimators        0.172035
max_depth           0.070712
min_samples_leaf    0.317756
max_features       -0.209077

	---precision_1---

                  precision_1
n_estimators         0.037999
max_depth            0.037534
min_samples_leaf    -0.681280
max_features        -0.299728

	---recall_1---

                  recall_1
n_estimators      0.000044
max_depth        -0.691022
min_samples_leaf -0.489642
max_features      0.008182


	Best setups per metric

	---f1-score_1---

     f1-score_1  n_estimators  max_depth  min_samples_leaf  max_features   
98     0.513274            10        120                 6           0.1  \
77     0.513274            10         80                 6           0.1   
171    0.507463            40         20                10           0.4   
337    0.504348            80        120                 6           0.1   
533 

## SMOTE

In [92]:
run_inference(data=(X_rose, X_test, Y_rose, Y_test),
              base_clf=RandomForestClassifier,
              params=rf_params2,
              metrics=('f1-score', 'precision', 'recall'),
              classes=(0, 1),
              output_file=os.path.join('data', '.local', 'RF_SMOTE_results.csv'),
              multiprocessing_mode=True)

Number of classifiers to be trained: 600


Multiprocessing method: 100%|█████████████████| 600/600 [02:04<00:00,  4.82it/s]

Results writen into data/.local/RF_SMOTE_results.csv.
@timer:<function processes at 0x7f5bfd83cd30> exec. time: 124.69 s.


In [96]:
rf_smote_res = pd.read_csv(os.path.join('data', '.local', 'RF_SMOTE_results.csv'), sep=';')
rf_smote_res

,n_estimators,criterion,max_depth,min_samples_leaf,max_features,f1-score_0,f1-score_1,precision_0,precision_1,recall_0,recall_1
0,10,entropy,3,2,0.10,0.913355,0.241546,0.993994,0.141243,0.844819,0.833333
1,10,entropy,3,6,0.10,0.913355,0.241546,0.993994,0.141243,0.844819,0.833333
2,10,entropy,3,4,0.10,0.913355,0.241546,0.993994,0.141243,0.844819,0.833333
3,10,entropy,3,2,0.15,0.923499,0.286445,0.997630,0.169184,0.859622,0.933333
4,10,entropy,3,4,0.15,0.923499,0.286445,0.997630,0.169184,0.859622,0.933333
...,...,...,...,...,...,...,...,...,...,...,...
595,200,entropy,120,6,0.40,0.983682,0.448276,0.982680,0.464286,0.984686,0.433333
596,200,entropy,120,12,0.25,0.980231,0.461538,0.986054,0.397590,0.974477,0.550000
597,200,entropy,120,8,0.40,0.982860,0.480620,0.985128,0.449275,0.980602,0.516667
598,200,entropy,120,10,0.40,0.981045,0.447761,0.984576,0.405405,0.977540,0.500000


In [97]:
generate_hprs_report(df=rf_smote_res, hprs=HPRS, metrics=['f1-score_1', 'precision_1', 'recall_1'])

		Hyperparameters Report

	Correlation between given metrics and hyperparameters

	---f1-score_1---

                  f1-score_1
n_estimators        0.202248
max_depth           0.025263
min_samples_leaf    0.274663
max_features       -0.119780

	---precision_1---

                  precision_1
n_estimators        -0.030351
max_depth            0.003941
min_samples_leaf    -0.711226
max_features        -0.323643

	---recall_1---

                  recall_1
n_estimators     -0.011579
max_depth        -0.676493
min_samples_leaf -0.508793
max_features     -0.022411


	Best setups per metric

	---f1-score_1---

     f1-score_1  n_estimators  max_depth  min_samples_leaf  max_features   
226    0.510638            40        120                10          0.10  \
394    0.507692           120         20                10          0.10   
203    0.503937            40         80                10          0.10   
480    0.503497           120        120                12          0.25   
179 

## Weights

In [136]:
rf_params_wght = {
    'n_estimators': [40, 80],
    'criterion': ['entropy'],
    'max_depth': [3, 20, 40, 80],
    'min_samples_leaf': [8, 12],
    'max_features': [0.1, 0.25, 0.4],
    'class_weight': [
        {0: 1, 1: 2},
        {0: 1, 1: 4},
        {0: 1, 1: 8},
        {0: 1, 1: 12}
    ]
}

In [137]:
run_inference(data=(X_train, X_test, Y_train, Y_test),
              base_clf=RandomForestClassifier,
              params=rf_params_wght,
              metrics=('f1-score', 'precision', 'recall'),
              classes=(0, 1),
              output_file=os.path.join('data', '.local', 'RF_WEIGHTS_results.csv'),
              multiprocessing_mode=True)

Number of classifiers to be trained: 192


Multiprocessing method:  50%|█████████         | 96/192 [00:06<00:06, 15.79it/s]/home/maksym/Documents/venvs/Bankruptcy-Prediction/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/maksym/Documents/venvs/Bankruptcy-Prediction/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/maksym/Documents/venvs/Bankruptcy-Prediction/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels wi

Results writen into data/.local/RF_WEIGHTS_results.csv.
@timer:<function processes at 0x7f5bfd83cd30> exec. time: 18.47 s.


In [ ]:
rf_weights_res = pd.read_csv(os.path.join('data', '.local', 'RF_WEIGHTS_results.csv'), sep=';')
rf_weights_res

# XGBoost

In [95]:
xgb_params = {'n_estimators': [4, 8, 12, 36, 80, 120],
              'criterion': ['squared_error'],
              'max_depth': [3, 6, 12, 24, 48],
              'min_samples_leaf': [1, 4, 8, 12],
              'max_features': [round(0.1 * x, 1) for x in range(3, 6)],
             'learning_rate': [0.05, 0.1, 0.15]}
print("\n".join(re.split('(?<=]),', xgb_params.__str__())))
print("\nNumber of settups:", len(list(product(*list(xgb_params.values())))))

{'n_estimators': [4, 8, 12, 36, 80, 120]
 'criterion': ['squared_error']
 'max_depth': [3, 6, 12, 24, 48]
 'min_samples_leaf': [1, 4, 8, 12]
 'max_features': [0.3, 0.4, 0.5]
 'learning_rate': [0.05, 0.1, 0.15]}

Number of settups: 1080


## ROSE

In [ ]:
run_inference(data=(X_rose, X_test, Y_rose, Y_test),
              base_clf=GradientBoostingClassifier,
              params=xgb_params,
              metrics=('f1-score', 'precision', 'recall'),
              classes=(0, 1),
              output_file=os.path.join('data', '.local', 'XGB_ROSE_results.csv'),
              multiprocessing_mode=True)

In [46]:
xgb_rose_res = pd.read_csv(os.path.join('data', '.local', 'XGB_ROSE_results.csv'), sep=';')
xgb_rose_res.head()

,n_estimators,criterion,max_depth,min_samples_leaf,max_features,learning_rate,f1-score_0,f1-score_1,precision_0,precision_1,recall_0,recall_1
0,4,squared_error,3,1,0.3,0.15,0.916348,0.194737,0.992303,0.112121,0.851193,0.74
1,4,squared_error,3,1,0.3,0.05,0.899277,0.184685,0.994462,0.104061,0.820721,0.82
2,4,squared_error,3,4,0.3,0.05,0.899277,0.184685,0.994462,0.104061,0.820721,0.82
3,4,squared_error,3,1,0.3,0.10,0.908439,0.194175,0.993965,0.110497,0.836465,0.80
4,4,squared_error,3,4,0.3,0.10,0.908439,0.194175,0.993965,0.110497,0.836465,0.80


In [63]:
generate_hprs_report(df=xgb_rose_res, hprs=HPRS, metrics=['f1-score_1', 'precision_1', 'recall_1'])

		Hyperparameters Report

	Correlation between given metrics and hyperparameters

	---f1-score_1---

                  f1-score_1
n_estimators       -0.166515
max_depth           0.159156
min_samples_leaf    0.200320
max_features       -0.181393

	---precision_1---

                  precision_1
n_estimators         0.443215
max_depth            0.241017
min_samples_leaf    -0.193523
max_features        -0.173509

	---recall_1---

                  recall_1
n_estimators      0.072677
max_depth        -0.574675
min_samples_leaf -0.135224
max_features     -0.024279


	Best setups per metric

	---f1-score_1---

     f1-score_1  n_estimators  max_depth  min_samples_leaf  max_features   
779    0.421875            80          6                12           0.3  \
758    0.420000            80          6                 1           0.3   
284    0.418182             8         12                12           0.4   
636    0.416667            36         12                12           0.3   
710 

## SMOTE

In [94]:
run_inference(data=(X_rose, X_test, Y_rose, Y_test),
              base_clf=GradientBoostingClassifier,
              params=xgb_params,
              metrics=('f1-score', 'precision', 'recall'),
              classes=(0, 1),
              output_file=os.path.join('data', '.local', 'XGB_SMOTE_results.csv'),
              multiprocessing_mode=True)

NameError: name 'xgb_params' is not defined

In [ ]:
xgb_smote_res = pd.read_csv(os.path.join('data', '.local', 'XGB_SMOTE_results.csv'), sep=';')
xgb_smote_res.head()

In [ ]:
generate_hprs_report(df=xgb_smote_res, hprs=HPRS, metrics=['f1-score_1', 'precision_1', 'recall_1'])

# LightGBM

In [130]:
LGBM_HPRS = ['num_leaves', 'max_depth', 'learning_rate', 'n_estimators']
lgbm_params = {'num_leaves': [5, 10, 15, 20, 30, 50, 100],
               'max_depth': [1, 3, 5, 10, 20, 30, 50, 100],
               'learning_rate': [0.1, 0.2, 0.3],
               'n_estimators': [20, 50, 75, 100, 150, 200],
               'verbosity': [1]
              }

print("\n".join(re.split('(?<=]),', lgbm_params.__str__())))
print("\nNumber of settups:", len(list(product(*list(lgbm_params.values())))))

{'num_leaves': [5, 10, 15, 20, 30, 50, 100]
 'max_depth': [1, 3, 5, 10, 20, 30, 50, 100]
 'learning_rate': [0.1, 0.2, 0.3]
 'n_estimators': [20, 50, 75, 100, 150, 200]
 'verbosity': [1]}

Number of settups: 1008


## ROSE

In [ ]:
run_inference(data=(X_rose, X_test, Y_rose, Y_test),
              base_clf=LGBMClassifier,
              params=lgbm_params,
              metrics=('f1-score', 'precision', 'recall'),
              classes=(0, 1),
              output_file=os.path.join('data', '.local', 'LGBM_ROSE_results.csv'),
              multiprocessing_mode=False)

In [132]:
lgbm_rose_res = pd.read_csv(os.path.join('data', '.local', 'LGBM_ROSE_results.csv'), sep=';')
lgbm_rose_res

,num_leaves,max_depth,learning_rate,n_estimators,verbosity,f1-score_0,f1-score_1,precision_0,precision_1,recall_0,recall_1
0,5,1,0.1,20,1,0.890647,0.224449,0.990572,0.130841,0.809035,0.788732
1,5,1,0.1,50,1,0.913116,0.259434,0.990396,0.155807,0.847023,0.774648
2,5,1,0.1,75,1,0.917310,0.268293,0.990476,0.162242,0.854209,0.774648
3,5,1,0.1,100,1,0.922064,0.279188,0.990566,0.170279,0.862423,0.774648
4,5,1,0.1,150,1,0.928825,0.296496,0.990692,0.183333,0.874230,0.774648
...,...,...,...,...,...,...,...,...,...,...,...
1003,100,100,0.3,50,1,0.982234,0.285714,0.971386,0.518519,0.993326,0.197183
1004,100,100,0.3,75,1,0.982750,0.291667,0.971414,0.560000,0.994353,0.197183
1005,100,100,0.3,100,1,0.982750,0.291667,0.971414,0.560000,0.994353,0.197183
1006,100,100,0.3,150,1,0.982750,0.291667,0.971414,0.560000,0.994353,0.197183


In [133]:
generate_hprs_report(df=lgbm_rose_res, hprs=LGBM_HPRS, metrics=['f1-score_1', 'precision_1', 'recall_1'])

		Hyperparameters Report

	Correlation between given metrics and hyperparameters

	---f1-score_1---

               f1-score_1
num_leaves      -0.021802
max_depth        0.042055
learning_rate   -0.030831
n_estimators    -0.136360

	---precision_1---

               precision_1
num_leaves        0.306589
max_depth         0.094902
learning_rate     0.180264
n_estimators      0.149255

	---recall_1---

               recall_1
num_leaves     0.067572
max_depth     -0.322722
learning_rate -0.048952
n_estimators  -0.099746


	Best setups per metric

	---f1-score_1---

     f1-score_1  num_leaves  max_depth  learning_rate  n_estimators   
685    0.455172          30         50            0.1            50  \
667    0.455172          30         30            0.1            50   
703    0.455172          30        100            0.1            50   
649    0.455172          30         20            0.1            50   
930    0.444444         100         10            0.3            20   

  

## SMOTE

In [ ]:
run_inference(data=(X_rsmote, X_test, Y_smote, Y_test),
              base_clf=LGBMClassifier,
              params=lgbm_params,
              metrics=('f1-score', 'precision', 'recall'),
              classes=(0, 1),
              output_file=os.path.join('data', '.local', 'LGBM_SMOTE_results.csv'),
              multiprocessing_mode=False)

In [ ]:
lgbm_smote_res = pd.read_csv(os.path.join('data', '.local', 'LGBM_SMOTE_results.csv'), sep=';')
lgbm_smote_res

## Weights

In [135]:
LGBM_WGHTS_HPRS = ['num_leaves', 'max_depth', 'learning_rate', 'n_estimators', 'class_weight']
lgbm_wghts_params = {'num_leaves': [5, 10, 15, 20, 30, 50, 100],
               'max_depth': [1, 3, 5, 10,  30, 50, 100],
               'learning_rate': [0.1, 0.2, 0.3],
               'n_estimators': [20, 50, 100, 150, 200],
                'class_weight': [
                    {0: 1, 1: 2},
                    {0: 1, 1: 4},
                    {0: 1, 1: 8},
                    {0: 1, 1: 12}
                                ]
              }

print("\n".join(re.split('(?<=]),', lgbm_wghts_params.__str__())))
print("\nNumber of settups:", len(list(product(*list(lgbm_wghts_params.values())))))

{'num_leaves': [5, 10, 15, 20, 30, 50, 100]
 'max_depth': [1, 3, 5, 10, 30, 50, 100]
 'learning_rate': [0.1, 0.2, 0.3]
 'n_estimators': [20, 50, 100, 150, 200]
 'class_weight': [{0: 1, 1: 2}, {0: 1, 1: 4}, {0: 1, 1: 8}, {0: 1, 1: 12}]}

Number of settups: 2940


In [ ]:
run_inference(data=(X_rsmote, X_test, Y_smote, Y_test),
              base_clf=LGBMClassifier,
              params=lgbm_params,
              metrics=('f1-score', 'precision', 'recall'),
              classes=(0, 1),
              output_file=os.path.join('data', '.local', 'LGBM_WEIGHTS_results.csv'),
              multiprocessing_mode=False)

In [ ]:
lgbm_weights_res = pd.read_csv(os.path.join('data', '.local', 'LGBM_WEIGHTS_results.csv'), sep=';')
lgbm_weights_res